In [2]:
text="""Artificial intelligence is transforming the world at an unprecedented pace, revolutionizing industries and reshaping the way humans interact with technology.
From healthcare to finance, AI-powered systems are improving efficiency, accuracy, and decision-making.
In healthcare, AI assists in diagnosing diseases, predicting patient outcomes, and personalizing treatments based on vast amounts of data.
Medical imaging powered by deep learning can detect anomalies such as tumors or fractures with higher precision than human radiologists.
AI-driven robotic surgeries enhance precision and reduce risks, improving patient recovery times.
In finance, algorithms analyze market trends, detect fraudulent transactions, and automate trading strategies.
The rise of machine learning and deep learning has enabled computers to learn from data and make predictions without explicit programming.
Banks and financial institutions use AI to assess creditworthiness, manage risks, and provide personalized financial recommendations.
In the realm of communication, natural language processing allows AI models to understand and generate human-like text, revolutionizing customer service with chatbots and virtual assistants.
AI-driven translation services break language barriers, enabling global connectivity.
Social media platforms leverage AI for content moderation, detecting harmful speech, and curating personalized content for users.
E-commerce platforms use AI-powered recommendation engines to suggest products based on user behavior, enhancing shopping experiences and boosting sales.
In transportation, autonomous vehicles leverage AI to navigate roads, interpret traffic signals, and enhance safety through real-time decision-making.
AI-powered traffic management systems optimize road conditions by predicting congestion patterns and adjusting signals accordingly.
Education is also undergoing a transformation with AI-driven learning platforms that provide personalized tutoring, automate grading, and analyze student progress to offer targeted learning resources.
AI can identify struggling students and suggest interventions to improve learning outcomes.
In entertainment, AI is used for content generation, music composition, and video editing.
Streaming services analyze user preferences to recommend movies, TV shows, and music.
AI-generated art and literature challenge traditional notions of creativity and human expression.
While AI offers numerous benefits, ethical concerns arise regarding job displacement, privacy, and bias in decision-making algorithms.
Automation threatens to replace human workers in industries such as manufacturing, logistics, and customer service, raising questions about workforce reskilling and economic inequality.
AI systems trained on biased datasets may produce unfair outcomes, perpetuating discrimination in areas such as hiring, lending, and law enforcement.
The development of explainable AI is crucial to ensure transparency and fairness in automated systems.
Researchers are working on methods to make AI decisions more interpretable, reducing the black-box nature of deep learning models.
Governments and organizations are working to establish regulations that balance innovation with accountability, ensuring AI is used ethically and responsibly.
As AI continues to evolve, its integration into daily life will expand, influencing education, entertainment, and scientific research.
AI is being used to accelerate discoveries in fields such as drug development, climate modeling, and space exploration.
Scientists use AI to analyze complex genetic data, leading to breakthroughs in precision medicine and personalized therapies.
In climate science, AI helps model and predict climate change effects, guiding policy decisions for sustainability.
The use of AI in robotics is enabling the creation of humanoid robots that can perform tasks ranging from elderly care to industrial automation.
The future of AI depends on responsible development and collaboration between technologists, policymakers, and society.
While AI presents unprecedented opportunities, ensuring its ethical use requires ongoing discussion, regulation, and adaptation.
By leveraging AI responsibly, humanity can unlock its full potential while mitigating risks and ensuring a future where technology serves all of society."""



In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
T=Tokenizer()
T.fit_on_texts([text])


In [7]:
len(T.word_index)

346

In [20]:
inputs=[]
for sentence in text.split('.'):
  token=T.texts_to_sequences([sentence])
  for i in range(1,len(token[0])):
    inputs.append((token[0][:i+1]))

In [25]:
max_len=max([len(x) for x in inputs])
max_len

26

In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input=pad_sequences(inputs,maxlen=max_len,padding='pre')


In [28]:
len(padded_input[0])

26

In [29]:
x=padded_input[:,:-1]
y=padded_input[:,-1]

In [31]:
x.shape


(552, 25)

In [32]:
y.shape

(552,)

In [34]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=len(T.word_index)+1)

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,Dropout


In [57]:
model=Sequential()
model.add(Embedding(input_dim=len(T.word_index)+1, output_dim=100, input_length=26 - 1))
model.build((None,26-1)) # build the Embedding to inilizices the weight
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(len(T.word_index)+1,activation='softmax'))

In [58]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [59]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ (None, 25, 100)             │          34,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 25, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 100)                 │         100,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 347)                 │          35,047 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,747 (1.22 MB)

 Trainable params: 320,747 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
h=model.fit(x,y,epochs=100,verbose=0)

In [62]:
import numpy as np

In [82]:
test1="The combination of deep learning and reinforcement learning"
for i in range(10):
  tokenized_text=T.texts_to_sequences([test1])
  padded_text=pad_sequences(tokenized_text,maxlen=25,padding='pre')
  pos=np.argmax(model.predict(padded_text))
  for word,index in T.word_index.items():
    if (index==pos):
      test1=test1+" "+word
      print(test1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
The combination of deep learning and reinforcement learning learning
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
The combination of deep learning and reinforcement learning learning learning
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
The combination of deep learning and reinforcement learning learning learning as
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
The combination of deep learning and reinforcement learning learning learning as as
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
The combination of deep learning and reinforcement learning learning learning as as manufacturing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
The combination of deep learning and reinforcement learning learning learning as as manufacturing logistics
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
The combination of deep learning and reinforcement learning learning learning as as manufacturing logistics and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
The combination of deep learning and reinforcement learning learni

In [83]:
test1="Artificial intelligence is transforming the way"
for i in range(10):
  tokenized_text=T.texts_to_sequences([test1])
  padded_text=pad_sequences(tokenized_text,maxlen=25,padding='pre')
  pos=np.argmax(model.predict(padded_text))
  for word,index in T.word_index.items():
    if (index==pos):
      test1=test1+" "+word
      print(test1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Artificial intelligence is transforming the way at
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Artificial intelligence is transforming the way at an
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Artificial intelligence is transforming the way at an unprecedented
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Artificial intelligence is transforming the way at an unprecedented pace
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Artificial intelligence is transforming the way at an unprecedented pace revolutionizing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Artificial intelligence is transforming the way at an unprecedented pace revolutionizing industries
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Artificial intelligence is transforming the way at an unprecedented pace revolutionizing industries and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Artificial intelligence is transforming the way at an unprecedented pace revolutionizing industries and reshaping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Artif

In [85]:
test1 = "Machine learning algorithms are designed to"
for i in range(10):
  tokenized_text=T.texts_to_sequences([test1])
  padded_text=pad_sequences(tokenized_text,maxlen=25,padding='pre')
  pos=np.argmax(model.predict(padded_text))
  for word,index in T.word_index.items():
    if (index==pos):
      test1=test1+" "+word
      print(test1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Machine learning algorithms are designed to model
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Machine learning algorithms are designed to model and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Machine learning algorithms are designed to model and predict
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Machine learning algorithms are designed to model and predict climate
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Machine learning algorithms are designed to model and predict climate change
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Machine learning algorithms are designed to model and predict climate change effects
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Machine learning algorithms are designed to model and predict climate change effects policy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Machine learning algorithms are designed to model and predict climate change effects policy decisions
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Machine learning algorithms are designed to model and predict clim

In [88]:
test1 = "Deep learning models such as convolutional neural networks"
for i in range(10):
  tokenized_text=T.texts_to_sequences([test1])
  padded_text=pad_sequences(tokenized_text,maxlen=25,padding='pre')
  pos=np.argmax(model.predict(padded_text))
  for word,index in T.word_index.items():
    if (index==pos):
      test1=test1+" "+word
      print(test1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Deep learning models such as convolutional neural networks in
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Deep learning models such as convolutional neural networks in recommendation
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Deep learning models such as convolutional neural networks in recommendation engines
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Deep learning models such as convolutional neural networks in recommendation engines and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Deep learning models such as convolutional neural networks in recommendation engines and suggest
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Deep learning models such as convolutional neural networks in recommendation engines and suggest products
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Deep learning models such as convolutional neural networks in recommendation engines and suggest products based
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Deep learning models such as convolutional neural networks in recomm